In [1]:
from pathlib import Path
import rasterio
import geopandas as gpd

data_dir = Path("../data")

# read DEM
dem = rasterio.open(data_dir / "srtm.tif")
# read NP boundary
zion_boundary = gpd.read_file(data_dir / "zion.gpkg")
# read points of interest
zion_points = gpd.read_file(data_dir / "zion_points.gpkg")

In [ ]:
# have a look at the data?
dem.read()

In [ ]:
zion_boundary

In [ ]:
zion_points.head()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

f, ax = plt.subplots()
flattened = dem.read().ravel()
ax.hist(flattened, bins=50)
ax.axvline(np.quantile(flattened, 0.95), ls='--', color='k', label="95th percentile")
ax.legend()
ax.set_xlabel("Elevation (m)")


In [ ]:
import rasterio.plot

# plot our DEM and boundary data
fig, ax = plt.subplots()
rasterio.plot.show(dem, ax=ax)
zion_boundary.to_crs(dem.crs).plot(ax=ax, color='none', edgecolor='black')

In [ ]:
# map the points
fig, ax = plt.subplots()
rasterio.plot.show(dem, ax=ax)
zion_points.plot(ax=ax, color='black')

In [8]:
import rasterstats

# raster extraction (points)
result1 = rasterstats.point_query(
    zion_points,
    dem.read(1),
    nodata = dem.nodata,
    affine = dem.transform,
    interpolate='nearest'
)
zion_points["height_m"] = result1

In [ ]:
# point height distribution
zion_points.height_m.plot(kind="hist")

In [ ]:
import shapely

# extract a transect
coords = [[-113.2, 37.45], [-112.9, 37.2]]
transect = shapely.LineString(coords)
transect_gdf = gpd.GeoSeries(transect, crs=4326)
print(transect_gdf)

In [ ]:
# project to CRS in meters
transect_proj_gdf = transect_gdf.to_crs(transect_gdf.estimate_utm_crs())
transect_proj = transect_proj_gdf.iloc[0]
print(transect_proj_gdf)

In [ ]:
# get 250m intervals of distance along transect
distances = np.arange(0, transect_proj.length, 250)
distances[:5]

In [ ]:
zion_transect_pnt = [transect_proj.interpolate(distance) for distance in distances]
zion_transect_pnt = gpd.GeoSeries(zion_transect_pnt, crs=transect_proj_gdf.crs).to_crs(dem.crs)
zion_transect_pnt

In [ ]:
import rasterstats

# sample the heights along the transect
result = rasterstats.point_query(
    zion_transect_pnt,
    dem.read(1),
    nodata = dem.nodata,
    affine = dem.transform,
    interpolate='nearest'
)
# save the result to a GeoDataFrame
zion_transect = gpd.GeoDataFrame(geometry=zion_transect_pnt.geometry)
zion_transect['dist'] = distances
zion_transect['elev'] = result
zion_transect

In [ ]:
f, ax = plt.subplots()
ax.plot(zion_transect.dist, zion_transect.elev)
ax.set_xlabel("Distance [m]")
ax.set_ylabel("Elevation [m]")
ax.grid()

In [ ]:
f, ax = plt.subplots()
rasterio.plot.show(dem, ax=ax)
zion_transect.plot(ax=ax, color='white')
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")